<h1>Load Data</h1>

In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt

### beers and breweries

In [7]:
beer_file = r'../data/bigData/beers.csv'
breweries_file = r'../data/bigData/breweries.csv'

beers = pd.read_csv (beer_file)

breweries = pd.read_csv (breweries_file)
breweries.rename(columns={'id':'brewery_id'},inplace=True)

reviews = pd.read_csv (r"../data/bigData/reviews.csv")

### long and lat of cities

In [8]:
cities_pos_file  = r'lat_long_03-05-2020_19:21.csv' 

cities_pos = pd.read_csv (cities_pos_file)

### join_beer_brewery

In [9]:
beer_and_breweries = pd.merge(beers,breweries,on="brewery_id")
beer_and_breweries.rename(columns={'name_x':'beer','name_y':'brewery','country_y':'country'},inplace=True)
beer_and_breweries = beer_and_breweries[["id","beer","style","availability","abv","retired","brewery","city","country","types"]]


### cleaning

In [10]:
beer_and_breweries = beer_and_breweries[beer_and_breweries["abv"]<70]

Delete redondant data

In [11]:
del beers
del breweries

### print data infos

In [12]:
beer_and_breweries.head(2)

,id,beer,style,availability,abv,retired,brewery,city,country,types
0,202522,Olde Cogitator,English Oatmeal Stout,Rotating,7.3,f,Main Street Brewery,Pleasanton,US,"Brewery, Bar, Eatery"
1,8677,Red Leaf Strong Ale,American Strong Ale,Year-round,8.6,f,Main Street Brewery,Pleasanton,US,"Brewery, Bar, Eatery"


In [13]:
cities_pos.head(2)

,Country,City,lat,long
0,BE,Erpe-Mere,50.93604,4.03550
1,US,Thousand Palms,33.82002,-116.39029


In [14]:
reviews.head(2)

,beer_id,username,date,text,look,smell,taste,feel,overall,score
0,271781,bluejacket74,2017-03-17,"750 ml bottle, 2016 vintage, bottle #304 of...",4.0,4.0,4.0,4.25,4.0,4.03
1,125646,_dirty_,2017-12-21,,4.5,4.5,4.5,4.50,4.5,4.50


---
# Raw data to decent json

In [65]:
beer_and_breweries.set_index("id").to_json("../data/raw_data.json",orient='records')

In [67]:
bpc = beer_and_breweries.groupby("country").apply(lambda x: x.to_json(orient='records'))
bpc.to_json("../data/raw_data_per_c.json",orient="index")

---
# Infos per country

In [38]:
countries = beer_and_breweries[["country"]].drop_duplicates()
len(countries)

187

### nb_of_beers

In [39]:
temp = beer_and_breweries[["country","id"]].groupby("country").count()
temp.rename(columns={'id':'N_beer'},inplace=True)
countries = pd.merge(countries,temp,how='left',left_on='country',right_on='country')
del temp
countries.head()

,country,N_beer
0,US,231940.0
1,NO,1095.0
2,JP,1582.0
3,IT,2523.0
4,CA,20053.0


### Average abv/beer

In [40]:
temp = beer_and_breweries[["country","abv"]].groupby("country").mean()
temp.rename(columns={'abv':'avg_abv'},inplace=True)
temp.head()

,avg_abv
country,
AD,6.188889
AE,4.690000
AG,5.200000
AL,4.700000
AM,5.087500


In [41]:
countries = pd.merge(countries,temp,how='left',left_on='country',right_on='country')
del temp
countries.head()

,country,N_beer,avg_abv
0,US,231940.0,6.721202
1,NO,1095.0,6.403205
2,JP,1582.0,5.682332
3,IT,2523.0,6.400460
4,CA,20053.0,6.096917


### median abv

In [42]:
temp = beer_and_breweries[["country","abv"]].groupby("country").median()
temp.rename(columns={'abv':'median_abv'},inplace=True)
temp.head()

,median_abv
country,
AD,5.60
AE,4.69
AG,5.20
AL,4.30
AM,4.85


In [43]:
countries = pd.merge(countries,temp,how='left',left_on='country',right_on='country')
del temp
countries.head()

,country,N_beer,avg_abv,median_abv
0,US,231940.0,6.721202,6.3
1,NO,1095.0,6.403205,6.0
2,JP,1582.0,5.682332,5.0
3,IT,2523.0,6.400460,6.0
4,CA,20053.0,6.096917,5.6


### most abv

In [44]:
temp = beer_and_breweries[["country","abv"]].groupby("country").max()
temp.rename(columns={'abv':'max_abv'},inplace=True)
temp.head()

,max_abv
country,
AD,8.50
AE,4.69
AG,5.60
AL,6.50
AM,8.50


In [45]:
countries = pd.merge(countries,temp,how='left',left_on='country',right_on='country')
del temp
countries.head()

,country,N_beer,avg_abv,median_abv,max_abv
0,US,231940.0,6.721202,6.3,56.0
1,NO,1095.0,6.403205,6.0,20.0
2,JP,1582.0,5.682332,5.0,15.0
3,IT,2523.0,6.400460,6.0,16.0
4,CA,20053.0,6.096917,5.6,66.0


### number of breweries

In [54]:
temp = beer_and_breweries[["country","brewery"]].groupby("country").nunique()[["brewery"]]
temp.rename(columns={'brewery':'n_brewery'},inplace=True)
temp.head()

,n_brewery
country,
AD,1
AE,1
AG,1
AL,7
AM,4


In [55]:
countries = pd.merge(countries,temp,how='left',left_on='country',right_on='country')
del temp
countries.head()

,country,N_beer,avg_abv,median_abv,max_abv,n_brewery
0,US,231940.0,6.721202,6.3,56.0,7338.0
1,NO,1095.0,6.403205,6.0,20.0,77.0
2,JP,1582.0,5.682332,5.0,15.0,164.0
3,IT,2523.0,6.400460,6.0,16.0,367.0
4,CA,20053.0,6.096917,5.6,66.0,940.0


## Saving to csv and json

In [60]:
countries=countries.set_index("country")
countries.head()

,N_beer,avg_abv,median_abv,max_abv,n_brewery
country,,,,,
US,231940.0,6.721202,6.3,56.0,7338.0
NO,1095.0,6.403205,6.0,20.0,77.0
JP,1582.0,5.682332,5.0,15.0,164.0
IT,2523.0,6.400460,6.0,16.0,367.0
CA,20053.0,6.096917,5.6,66.0,940.0


In [61]:
countries.to_csv(r"../data/countries_basic_data.csv")

In [63]:

countries.to_json(r"../data/countries_basic_data.json",orient='index')

---
# Infos per city

---
# Draft

In [37]:
beer_and_breweries.loc[beer_and_breweries["abv"].idxmax()]

id                              104620
beer                       Snake Venom
style               British Barleywine
availability     Limited (brewed once)
abv                               67.5
retired                              t
brewery                    Brewmeister
city                            Aboyne
country                             GB
types                          Brewery
Name: 325099, dtype: object

## Beer types

In [16]:
beer_and_breweries.columns

Index(['id', 'beer', 'style', 'availability', 'abv', 'retired', 'brewery',
       'city', 'country', 'types'],
      dtype='object')

In [23]:
beer_types = beer_and_breweries[['style','id']].groupby('style').agg('count').reset_index()

In [59]:
IPA = beer_types[beer_types['style'].str.contains('IPA')]

In [136]:
Lager = beer_types[(beer_types['style'].str.contains('Lager'))]

In [137]:
Ale = beer_types[(beer_types['style'].str.contains('Ale')) | (beer_types['style'].str.contains('Gose'))]

In [138]:
Stout = beer_types[(beer_types['style'].str.contains('Stout'))]

In [139]:
Pilsner = beer_types[beer_types['style'].str.contains('Pils')]

In [140]:
Porter = beer_types[beer_types['style'].str.contains('Porter')]

In [141]:
Fruit = beer_types[beer_types['style'].str.contains('Fruit')]

In [142]:
Roggenbier = beer_types[(beer_types['style'].str.contains('Roggen'))|(beer_types['style'].str.contains('Rye'))]

In [143]:
Herb = beer_types[(beer_types['style'].str.contains('Herb'))]

In [144]:
Smoke = beer_types[(beer_types['style'].str.contains('Smoke'))]

In [149]:
Barleywine=beer_types[(beer_types['style'].str.contains('Barleywine'))]

In [52]:
beer_types['style'].to_list()

['American Adjunct Lager',
 'American Amber / Red Ale',
 'American Amber / Red Lager',
 'American Barleywine',
 'American Black Ale',
 'American Blonde Ale',
 'American Brett',
 'American Brown Ale',
 'American Brut IPA',
 'American Cream Ale',
 'American Dark Wheat Ale',
 'American IPA',
 'American Imperial IPA',
 'American Imperial Pilsner',
 'American Imperial Porter',
 'American Imperial Red Ale',
 'American Imperial Stout',
 'American Lager',
 'American Light Lager',
 'American Malt Liquor',
 'American Pale Ale (APA)',
 'American Pale Wheat Ale',
 'American Porter',
 'American Stout',
 'American Strong Ale',
 'American Wheatwine Ale',
 'American Wild Ale',
 'Baltic Porter',
 'Belgian Blonde Ale',
 'Belgian Dark Ale',
 'Belgian Dubbel',
 'Belgian Faro',
 'Belgian Fruit Lambic',
 'Belgian Gueuze',
 'Belgian IPA',
 'Belgian Lambic',
 'Belgian Pale Ale',
 'Belgian Quadrupel (Quad)',
 'Belgian Saison',
 'Belgian Strong Dark Ale',
 'Belgian Strong Pale Ale',
 'Belgian Tripel',
 'Belgian

,style,id
3,American Barleywine,2967
47,British Barleywine,1442
